# Catalan Number Generator
This notebook will explore how to generate catalan numbers and their respective sets

# First off we need to define some basic terminology used:
## Dyck words 
Words that are formed out of a binary alphabet and have a balance between the two characters (the same ammount of each) They are generated using a recursive implementation of the ballot problem.
## Catalan Numbers
Sequence of natural numbers that occur in various counting problems regarding recursive objects. In this notebook we are specifically iterpreting them in binary format by generating the different permutations of length (n) in which all are dyck words.
## Catalan Set
The set catalan numbers that are of length (n), all of them are dyck words.
## Balanced Parenthesis
A problem that seeks to verify if an expression has a balance of parenthesis and all parenthesis math each other. To have a balanced parenthesis expression it need to follow the ballot problem rule of having a higher frequency of opening parenthesis than closing ones at each segment starting from the start.

# Our external libraries used are used as follows:
* randint - used to generate a specific random dyckword of a given length
* default_timer - used to calculate the time taken between function calls
* unpack - used to unpack bytes from a file and convert them to a string of binary digits.

In [ ]:
from random import randint
from timeit import default_timer as timer
from struct import unpack

In [ ]:
def randomDyckWord(output, open, close, pairs):
  if open == pairs and close == pairs:
    return output
  else:
    to_be_appended = randint(0, 1)
    if to_be_appended == 1 and open < pairs:
      return randomDyckWord(output+'1', open+1, close, pairs)
    elif to_be_appended == 0 and close < open:
      return randomDyckWord(output+'0', open, close+1, pairs)
    else:
      return randomDyckWord(output, open, close, pairs)

input: empty string (output), integer 0 (open y close), integer which indicates the number of pair of 0 and 1 to use in the permutation (pairs).

funcion: Output all posible permutations (string) that can be made with a given number of pairs of 1 and 0 (pairs). 

In [ ]:
def dyckWords(output, open, close, pairs):
    if open == pairs and close == pairs:
        yield output
    else:
        if open < pairs:
          yield from dyckWords(output+'1', open+1, close, pairs)
        if close < open:
          yield from dyckWords(output+'0', open, close+1, pairs)

In [ ]:
def areParenthesisBalanced(expr): 
    stack = []  
    for char in expr: 
        if char == '1':   
            stack.append(char) 
        else:
            if not stack: 
                return False
            current_char = stack.pop() 
            if current_char == '1': 
                if char != "0": 
                    return False
    if stack: 
        return False
    return True

Hace una llave mas pequeña

In [ ]:
def retrieve_pairs(expr, pairs):
  sub_key = str()
  ones = 0
  zeroes = 0
  for char in expr:
    if char == '1' and ones < pairs:
      sub_key += '1'
      ones += 1
    elif char == '0' and zeroes < ones:
      sub_key += '0'
      zeroes += 1
  return sub_key

In [ ]:
retrieve_pairs("110101101000", 4)

'11010100'

In [ ]:
def stack_encrypt(text, key):
  cypher_text = str()
  text_lst = [str(char) for char in text]
  stack = list()
  for op in key:
    if op == '1':
      stack.append(text_lst.pop(0))
    if op == '0':
      cypher_text += stack.pop()
  return cypher_text

In [ ]:
def stack_decrypt(cypher, key):
  inverted_key = "".join([str(int(not int(i))) for i in key])[::-1]
  cypher_lst = [str(char) for char in cypher][::-1]
  decyphered_text = str()
  stack = list()
  for op in inverted_key:
    if op == '1':
      stack.append(cypher_lst.pop(0))
    if op == '0':
      decyphered_text += stack.pop()
  return decyphered_text[::-1]

# How full text encryption is achieved
To be able to encrypt successfully we need a function that is deterministic in nature and is symmetrical to its counter-part. The encryption process is divided in different cases according to the relationship of the length of the key and the length of the text that is going to be encrypted.
Cases:
* if key pairs is less than or equal to the length of the text
    * if the key pairs fit exactly inside the length of the text
        * if the key pairs fit exactly inside the text, no further processing of the key is needed, and therefore the text is divided into segments of equal size to the amount of pairs in the key, these segments will be encrypted individually and summed up, it may be the case that the number of pairs and the length of the text are the same in which case, no segmebtation occurs.
    * if the key pairs don't fit exactly inside the length of the text
        * if the key pairs don't fit exactly inside, then that means there will be a residual segment left over. IE. len_text % half_key_len = 4, then there will be a residual of 4 characters left over to encrypt. This special segment requires special treatment and requires that a new sub_key is generated such that it will be able to encrypt it successfully.
* if key pairs is greater than the length of the text
    * if key pairs is greater than the length of the text, the key needs to be processed in order for a sub_key to be created using the retrieve_pairs method. This key will be the exact size of double the size of the text.

In [ ]:
def catalan_encrypt(text, key):
  cypher_text = str()
  half_key_len = int(len(key)/2)
  if half_key_len <= len(text):
    # if key fits inside text
    if len(text) % half_key_len == 0:
      # if key fits perfectly inside text
      for i in range(0, len(text), half_key_len):
        segment = text[i:i+half_key_len]
        cypher_text += stack_encrypt(segment, key)
    else:
      # if key fits inside text but not perfectly
      special_segment_len = len(text) % half_key_len
      special_segment = text[-special_segment_len:]
      for i in range(0, len(text) - special_segment_len, half_key_len):
        segment = text[i:i+half_key_len]
        cypher_text += stack_encrypt(segment, key)
      sub_key = retrieve_pairs(key, special_segment_len)
      special_segment_cypher = stack_encrypt(special_segment, sub_key)
      cypher_text += special_segment_cypher
  elif half_key_len > len(text):
    sub_key = retrieve_pairs(key, len(text))
    cypher_text = stack_encrypt(text, sub_key)
    
  return cypher_text

# How full text decryption is achieved
Since these two functions are symmetrical in nature they are the same in structure. The only difference is the usage of the keys and sub_keys they are used in the stack_decrypt method instead of the stack_encrypt method, which guarantees the encryption and decryption of the text or segments there of.

In [ ]:
def catalan_decrypt(cypher, key):
  decyphered_text = str()
  half_key_len = int(len(key)/2)
  if half_key_len <= len(cypher):
    # if key fits inside text
    if len(cypher) % half_key_len == 0:
      # if key fits perfectly inside text
      for i in range(0, len(cypher), half_key_len):
        segment = cypher[i:i+half_key_len]
        decyphered_text += stack_decrypt(segment, key)
    else:
      # if key fits inside text but not perfectly
      special_segment_len = len(cypher) % half_key_len
      special_segment = cypher[-special_segment_len:]
      for i in range(0, len(cypher) - special_segment_len, half_key_len):
        segment = cypher[i:i+half_key_len]
        decyphered_text += stack_decrypt(segment, key)
      sub_key = retrieve_pairs(key, special_segment_len)
      special_segment_decyphered = stack_decrypt(special_segment, sub_key)
      decyphered_text += special_segment_decyphered
  elif half_key_len > len(cypher):
    sub_key = retrieve_pairs(key, len(cypher))
    decyphered_text = stack_decrypt(cypher, sub_key)
  return decyphered_text

# How the text file is read
The text file is oppened as read-only and in binary format to be able to truly scramble and jumble the text and not just create an anagram of the original text. Another useful thing about using binary is that we assert that with the bytes we are reading the length of the full joined text will be even. Evenness assures that when the encrytion is ran no segment of odd size is left over that can't be encrypted or decrypted using the stack encrypt and decrypt methods. To read the file byte by byte we unpack each byte in binary format using the unpack method from struct.

In [ ]:
binary_string = str()
with open("plainText.txt", "rb") as f:
  while (Byte := f.read(1)):
    print("{0:08b}".format(unpack("b", Byte)[0]))

01100001
01100010
01100011
01100100
01100101
01100110
00100000
01101000
01100101
01101100
01101100
01101111
00001010


In [ ]:
key_pairs = int(input("Please enter desired key length: "))
key = randomDyckWord('', 0, 0, key_pairs)
print(key)

Please enter desired key length: 4
10110010


In [ ]:
secret = catalan_encrypt(binary_string, key)
message = catalan_decrypt(secret, key)
print(secret, "\n" , message)

11000001110010001100100111000100110001011100110010000000110000101100010111000110110001101100111100001010 
 01100001011000100110001101100100011001010110011000100000011010000110010101101100011011000110111100001010
